In [1]:
import numpy as np
from PIL import Image
import glob

In [2]:
from ripser import ripser
from persim import plot_diagrams

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
img_names = glob.glob('images/*.png')

In [4]:
img = Image.open(img_names[0])
#img.show()

The function below takes snapshots of the plane that are translations apart. It takes an image (Image), the lens size (int), and the translation vector (tuple) that shifts the plane in the direction of the vector. The function returns a vector of images. 

In [5]:
def translation_snapshot(img, lens_sz, a):
    (x,y)=a
    x=-x #since x and y will shift the crop and not the plane, need to make sure it is going in right direction
    (length, height) = img.size
    (lens_length, lens_height) = lens_sz
    lens_corner_x=length/2.0 - lens_length/2.0 +x
    lens_corner_y=height/2.0 - lens_height/2.0 +y
    ret_img = img.crop((lens_corner_x,lens_corner_y, lens_corner_x+lens_length, lens_corner_y+lens_height))
        
    #make sure didn't fall off of the image
    (ret_length, ret_height) = ret_img.size
    if (lens_corner_x<0 or lens_corner_y<0 or lens_corner_x+lens_length>length or lens_corner_y+lens_height>height):
        print("lense went off image!")
        return False
    return ret_img

# returns a lense that is a percentage of a certain "bound" -- by default, this is half the image size, and the lense
# is one eighth of the bounding rectangle. Bound should be consistent across interval and lense generation.
# note that the lense can be a rectangle, used to have to be square. Also bound is currently centered around the middle
# of the image as a consequence of the translation_snapshot function, this should propably be changed.
def lense_sizer(img, bound =.5, lense_percentage=.125):
    (length, height) = img.size
    length = length * bound * lense_percentage
    height = height * bound * lense_percentage
    return (length, height)
    
# creates x and y intervals for the lense to travel across; these intervals are adjacent with no overlap and stay within
# the bounding rectangle. We may want to have overlap or gaps between the intervals if it makes better data.
def interval_sizer(img, lense_size, bound = .5):
    (length, height) = img.size
    (lense_length, lense_height) = lense_size
    
    intervalx = np.arange(bound * length * -0.5, bound * length * 0.5  ,lense_length)
    intervaly = np.arange(bound * height * -0.5, bound * height * 0.5  ,lense_height)
    
    return (intervalx, intervaly)

In [6]:
image_list = []
lens_sz = lense_sizer(img, .70, .25)
# note that linspace creates 10 even intervals, while arange creates intervals of distance 10 (given 10 as last arg)
degrees = np.arange(0, 360, 10)
(intervalx, intervaly) = interval_sizer(img, lens_sz, .75)
for d in degrees:
    rot = img.rotate(d)
    for x in intervalx:
        for y in intervaly:
            image_list.append(translation_snapshot(rot, lens_sz, (x,y)))

In [7]:
counter = 0

# lense size seems to be off right now
print(image_list[0])

for img in image_list:
    img.save("images/dataset/"+str(counter)+".png")
    counter+=1

<PIL.Image.Image image mode=RGBA size=34x34 at 0x267312EEB10>
